# /bin/bash

sudo pip install django-recommends 

python manage.py startapp myrecommend


# setting.py

INSTALLED_APPS = [
...........
........
.........
..........

]


에 

    'django.contrib.sites',

    'recommends',  
    'recommends.storages.djangoorm',

    'myrecommend',


4문장 추가 

SITE_ID = 1 이라는 옵션 추가



# model 정의

# /myrecommend/models.py

```python
from __future__ import unicode_literals
from django.db import models
from django.contrib.auth.models import User
from django.contrib.sites.models import Site
from django.utils.encoding import python_2_unicode_compatible


@python_2_unicode_compatible
class Product(models.Model):
    """A generic Product"""
    name = models.CharField(blank=True, max_length=100)
    sites = models.ManyToManyField(Site)

    def __str__(self):
        return self.name

    @models.permalink
    def get_absolute_url(self):
        return ('product_detail', [self.id])

    def sites_str(self):
        return ', '.join([s.name for s in self.sites.all()])
    sites_str.short_description = 'sites'


@python_2_unicode_compatible
class Vote(models.Model):
    """A Vote on a Product"""
    user = models.ForeignKey(User, related_name='votes')
    product = models.ForeignKey(Product)
    site = models.ForeignKey(Site)
    score = models.FloatField()

    def __str__(self):
        return "Vote"
```

# view 정의

# /myrecommend/views.py

```python

# Create your views here.
from django.contrib.auth import authenticate, login as _login
from django.http import HttpResponseRedirect

from .models import Product
from django.shortcuts import render

def login(request):
    username = request.POST['username']
    password = request.POST['password']
    user = authenticate(username=username, password=password)
    if user is not None:
        if user.is_active:
            _login(request, user)
            # redirect to a success page.
        else:
            # Return a 'disabled account' error message
            pass
    else:
        pass
        # Return an 'invalid login' error message.
    return HttpResponseRedirect(request.POST.get('next', '/'))


def ProductView(request,pk=None):
    object1=Product.objects.get(id=int(pk))
    return render(request, 'product_detail.html',{'object':object1})
    
```

# recommendations.py 생성
# /myrecommend/recommendations.py

# - 어떻게 돌아가는지는 자세히 모른다 ㅠㅠ

```python


#recommendations.py

from django.contrib.auth.models import User
from recommends.providers import RecommendationProvider
from recommends.providers import recommendation_registry

from .models import Product, Vote

class ProductRecommendationProvider(RecommendationProvider):
    def get_users(self):
        return User.objects.filter(is_active=True, votes__isnull=False).distinct()

    def get_items(self):
        return Product.objects.all()

    def get_ratings(self, obj):
        return Vote.objects.filter(product=obj)

    def get_rating_score(self, rating):
        return rating.score

    def get_rating_site(self, rating):
        return rating.site

    def get_rating_user(self, rating):
        return rating.user

    def get_rating_item(self, rating):
        return rating.product

recommendation_registry.register(Vote, [Product], ProductRecommendationProvider)


```

# url 정의
# /내프로젝트이름/urls.py

```python


#/myrecommend/urls.py


from django.conf.urls import url
from django.contrib import admin
from django.views.generic import TemplateView


urlpatterns = [
    url(r'^admin/', admin.site.urls),
    url(r'^login/','recommends.tests.views.login', name='login'),
    url(r'^product/(?P<pk>\d+)/$','myrecommend.views.ProductView', name='product_detail'),
    url(r'^$', TemplateView.as_view(template_name='home.html'), name='home'),
]

```

# admin 에 내 모델 추가

# /myrecommend/admin.py

```python
from django.contrib import admin

# Register your models here.

from myrecommend.models import Product,Vote

admin.site.register(Product)
admin.site.register(Vote)

```

# 일단 지금까지 잘됐는지 해보자!!


python manage.py makemigrations  
python manage.py migrate  

python manage.py createsuperuser  

슈퍼유저 생성하고~~~

어드민가서 확인해보자

확인해보자 모델 잘 생성됐는지.. c9에서할때 가끔씩 뻑날때가있다... 

# 모델은 잘생성됐다!! 모델에 fixture 넣어주고 탬플릿 추가하자



# 탬플릿 추가


# base.html 만들기

# /myrecommend/templates/base.html 

```python

<!DOCTYPE html>
<html>
<head>
	<meta charset="utf-8" />
	<title>Django Recommends</title>
</head>
<body>
	{% if user.is_anonymous %}
	<form action="{% url 'login' %}" method="post" id="login-form">{% csrf_token %}
	  <div class="form-row">
	    
	    <label for="id_username" class="required">Username:</label> <input id="id_username" type="text" name="username" maxlength="30">
	  </div>
	  <div class="form-row">
	    
	    <label for="id_password" class="required">Password:</label> <input type="password" name="password" id="id_password">
	  </div>
	  <div class="submit-row">
	    <label>&nbsp;</label><input type="submit" value="Log in">
	  </div>
	</form>
	{% else %}
	<p>Welcome, {{user.username}}. <a href="{% url 'admin:logout' %}">Logout</a>.</p>
	{% endif %}
	{% block content %}{% endblock %}
</body>
</html>

```

# home.html 만들기

# /myrecommend/templates/home.html 

```python
{% extends "base.html" %}
{% load recommends %}

{% block content %}
    {% suggested as suggestions limit 5  %}
	{% if user.is_authenticated %}
    {% if suggestions %}
	<h3>Suggestions for you:</h3>
	<ul>
	{% for suggestion in suggestions %}
		<li><a href="{{suggestion.object.get_absolute_url}}">{{suggestion.object}}</a></li>
	{% endfor %}
	</ul>

	{% endif %}
	{% endif %}
{% endblock %}

```

# product_detail.html 만들기

# /myrecommend/templates/product_detail.html  

```python
{% extends "base.html" %}
{% load recommends %}

{% block content %}
<h1>{{object.name}}</h1>
{% with object|similarities as similarities %}
{% if similarities %}
<ul>
	{% for similarity in similarities %}
	<li><a href="{{similarity.related_object.get_absolute_url}}">{{similarity.related_object}}</a></li>
	{% endfor %}
</ul>
{% endif %}
{% endwith %}
{% endblock %}

```

# fixture - 초기데이터 넣어주기

# myrecommend 앱에 fixture 폴더 추가하고 product.json 넣는다. product.json은 django-recommender github에 fixture 폴더안에 있음 

아 이때 product.json 이 테스트용 데이터라 데이터가 테스트용 모델로 설정돼있었다. 그래서 그안의 내용들을 컨트롤 f 바꾸기 신공으로 우리 모델과 맞게 바꿔줬다...

python manage.py loaddata myrecommend/fixture/products1.json  
python manage.py recommends_precompute


loaddata - fixture를 자신의 모델에 넣는명령어
recommends_precompute - django-recommend 깔면 추가되는 명령어로 이거하면 djangoorm 모델들 계산해준다.

djangoorm 모델에 Similarities db 가지고

새로운 유저가 들어올때 

reommendations.py 으로 계산을 해주면 되지 않을까..??

SITE DB도 어디쓰이는지모르겠음 ㅠ